In [1]:
# Install required libraries
!pip install transformers datasets accelerate bitsandbytes peft trl wandb huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# Login to Weights & Biases
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: deepdey524 (deepdey524-manipal-university-jaipur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os


In [5]:
# Load and preprocess the dataset
data = load_dataset("tatsu-lab/alpaca", split="train")
data_df = data.to_pandas()
data_df = data_df[:5000]  # Using first 5000 samples to stay within Colab limits
data_df["text"] = data_df.apply(lambda x: f"###Human: {x['instruction']} {x['input']} ###Assistant: {x['output']}", axis=1)
data = Dataset.from_pandas(data_df)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [6]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
# Enable 4-bit quantization to fit in Colab
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [8]:
# Load Mistral-7B with quantization
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=quantization_config,
    device_map="auto"
)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
# Prepare for training
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
# Apply LoRA fine-tuning
peft_config = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

In [11]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir="MISTRAL-7B-LLM-API-USE",
    per_device_train_batch_size=2,  # Lower batch size to fit in free T4 GPU
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=1,
    fp16=True,
    push_to_hub=True,
    report_to="wandb",
    max_steps=150
)

In [12]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    args=training_arguments,
    tokenizer=tokenizer
)


<ipython-input-12-0f1c38b83342>:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/5000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [13]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.753200
20,1.368500
30,1.301900
40,1.217300
50,1.192200
60,1.243300
70,1.136400
80,1.227400
90,1.175800
100,1.218000


TrainOutput(global_step=150, training_loss=1.2448005485534668, metrics={'train_runtime': 1116.465, 'train_samples_per_second': 1.075, 'train_steps_per_second': 0.134, 'total_flos': 5929056664043520.0, 'train_loss': 1.2448005485534668})

In [14]:
# Push the fine-tuned model to Hugging Face Hub
trainer.model.push_to_hub("MISTRAL-7B-LLM-API-USE")
tokenizer.push_to_hub("MISTRAL-7B-LLM-API-USE")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/deep0210/MISTRAL-7B-LLM-API-USE/commit/a8be24e35bf87d86967d9497b58ed281cce9f9dc', commit_message='Upload tokenizer', commit_description='', oid='a8be24e35bf87d86967d9497b58ed281cce9f9dc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/deep0210/MISTRAL-7B-LLM-API-USE', endpoint='https://huggingface.co', repo_type='model', repo_id='deep0210/MISTRAL-7B-LLM-API-USE'), pr_revision=None, pr_num=None)

In [17]:
from transformers import AutoConfig

MODEL_ID = "MISTRAL-7B-LLM-API-USE"

# Create config.json with correct model type
config = AutoConfig.from_pretrained("mistralai/Mistral-7B-v0.1")
config.save_pretrained(MODEL_ID)

# Push updated config to Hugging Face Hub
from huggingface_hub import HfApi
api = HfApi()
api.upload_folder(folder_path=MODEL_ID, repo_id="deep0210/MISTRAL-7B-LLM-API-USE")


optimizer.pt:   0%|          | 0.00/54.6M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/deep0210/MISTRAL-7B-LLM-API-USE/commit/cdd8c4f15d59b10a6c7df30e713d67868a63d5e2', commit_message='Upload folder using huggingface_hub', commit_description='', oid='cdd8c4f15d59b10a6c7df30e713d67868a63d5e2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/deep0210/MISTRAL-7B-LLM-API-USE', endpoint='https://huggingface.co', repo_type='model', repo_id='deep0210/MISTRAL-7B-LLM-API-USE'), pr_revision=None, pr_num=None)

In [18]:
print("Model fine-tuning complete and uploaded to Hugging Face!")

Model fine-tuning complete and uploaded to Hugging Face!
